In [1]:
import numpy as np
from scipy.spatial import distance
import math
import random
from copy import copy, deepcopy
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from bs4 import BeautifulSoup as bs
from numpy.linalg import norm
import time

In [2]:
inputs = []
with open('Data/topics.txt','r',encoding='utf-8') as file:
    for line in file:
        line = line[:-1]
        inputs.append(line)
    print(inputs)

['Coffee', 'Arduino', 'Anime', 'Astronomy', 'Biology', 'Chess', 'Cooking', 'Law', 'Space', 'Windows_Phone', 'Wood_Working']


In [3]:
def Text_Preprocessing(input_type):
    input_file = "Data/Training/"+ input_type + ".xml"
    array = []
    with open(input_file,'r',encoding='utf-8') as file:
        content = file.read()
        soup = bs(content,'lxml')
        for items in soup.findAll("row"):
            Document = dict()
            body = items['body']
               
            #removing links
            body = re.sub(r"<a.*</a>","", body)
            
            #removing tags
            body = re.sub("<.[^>]*>","", body)
            
            #removing unicode
            body = re.sub(r'[^\x00-\x7F]', ' ', body)
            
            #removing numbers
            body = re.sub(r'[-+]?\d+', '', body)

            #Lowercase the text
            body = body.lower()

            #Remove punctuations
            body = body.translate((str.maketrans('','',string.punctuation)))

            #Tokenize
            body = word_tokenize(body)

            #Remove stopwords
            stop_words = set(stopwords.words('english'))
            body = [word for word in body if not word in stop_words]

            #Lemmatize tokens
            lemmatizer = WordNetLemmatizer()
            body = [lemmatizer.lemmatize(word) for word in body]

            #Stemming tokens
            stemmer= PorterStemmer()
            body = [stemmer.stem(word) for word in body]
            
            if not body:
                continue
            
            Document[input_type] = body
            
            array.append(Document)
            #print(body, "\n\n\n")
        #print(array)
        return array

In [4]:
Training_set = []
# Validation_set = []
Test_set = []
for input_file in inputs:
    array = Text_Preprocessing(input_file)
    Training_set += array[:500]
#     Validation_set += array[500:500+200]
    Test_set += array[700:1200]

In [5]:
print(len(Training_set))
print(len(Validation_set))
print(len(Test_set))
# print(Training_set)

5500
2200
5500


In [7]:
# with open('TrainingSet.txt', 'w') as f:
#     for item in Training_set:
#         f.write("%s\n" % item)
        
        
# with open('Validation_set.txt', 'w') as f:
#     for item in Validation_set:
#         f.write("%s\n" % item)

FeatureSpace = []
with open('FeatureSpace.txt', 'r') as f:
    filecontents = f.readlines()
    for line in f:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        FeatureSpace.append(current_place)

In [5]:
FeatureSpace = []
for document in Training_set:
    temp = list(document.values())
    for i in temp[0]:
        if i not in FeatureSpace:
            FeatureSpace.append(i)
print(len(FeatureSpace))
# print(FeatureSpace)

18982


In [8]:
with open('FeatureSpace.txt', 'w') as f:
    for item in FeatureSpace:
        f.write("%s\n" % item)

In [9]:

Validation_set_S = random.choices(Validation_set, k = 600)

In [6]:
Y_train = []
X_train_HD = []
X_train_ED = []

# X_Validation_HD = []
# X_Validation_ED = []
# Y_Validation =[]

# X_test_HD = []
# X_test_ED = []
# Y_test = []

for i in range(len(Training_set)):
    Y_train.append(list(Training_set[i].keys())[0])
    temp = list(Training_set[i].values())[0]
    HD = []
    ED = []
    for i in range(len(FeatureSpace)):
        if FeatureSpace[i] in temp:
            HD.append(1)
        else:
            HD.append(0)
        
        ED.append(temp.count(FeatureSpace[i]))
        
    X_train_HD.append(HD)
    X_train_ED.append(ED)

# for i in range(len(Test_set)):
#     Y_test.append(list(Test_set[i].keys())[0])
#     temp = list(Test_set[i].values())[0]
#     HD = []
#     ED = []
#     for i in range(len(FeatureSpace)):
#         if FeatureSpace[i] in temp:
#             HD.append(1)
#         else:
#             HD.append(0)
        
#         ED.append(temp.count(FeatureSpace[i]))
        
#     X_test_HD.append(HD)
#     X_test_ED.append(ED)
    
    
# for i in range(len(Validation_set)):
#     Y_Validation.append(list(Validation_set[i].keys())[0])
#     temp = list(Validation_set[i].values())[0]
#     HD = []
#     ED = []
#     for i in range(len(FeatureSpace)):
#         if FeatureSpace[i] in temp:
#             HD.append(1)
#         else:
#             HD.append(0)
        
#         ED.append(temp.count(FeatureSpace[i]))
        
#     X_Validation_HD.append(HD)
#     X_Validation_ED.append(ED)

In [12]:
K = [1,3,5] 

In [1]:
print(len(X_Validation_HD[0]))

# print(X_train_HD[0])

NameError: name 'X_Validation_HD' is not defined

In [47]:
# f = open("File.txt", "a")
# f.write(Training_set)
# f.write("\n")
# f.write(Validation_set)
# f.write("\n")
# f.write(Test_set)
# f.close()

In [48]:
def getDistance(Dt,D1):
    count = 0
    for i in range(len(D1)):
        if D1[i] != Dt[i]:
            count += 1
    return count

In [49]:
def HammingDistance(Dt, X_train, Y_train):
    Hamming_D = []
    for i in range(len(X_train)):
#         Hamming_D.append(len(Dt)*distance.hamming(Dt, X_train[i]))
        Hamming_D.append(getDistance(Dt, X_train[i]))
    
    output = []
#     print(Hamming_D)
    for k in K:
        indices = sorted(range(len(Hamming_D)), key = lambda sub: Hamming_D[sub])[:k] 
        Y_output = []
        for i in indices:
            Y_output.append(Y_train[i])   
    
        output.append(max(set(Y_output), key = Y_output.count))
    
    return output

In [ ]:
def HammingDistance_Accuracy(X_Validation_HD, Y_Validation, X_train_HD, Y_train):
    count = [0,0,0]
    for i in range(len(X_Validation_HD)):
        result = HammingDistance(X_Validation_HD[i], X_train_HD, Y_train)
        
        for k in range(len(K)):
            if Y_Validation[i] == result[k] :
                count[k] += 1
        
        print(i, count)
        
    for k in range(len(K)):
        print("Accuracy is: ", (count[k]/len(Y_Validation) * 100))

# print(HammingDistance(X_test_HD[1200], X_train_HD, Y_train))
# print(Y_test[1200])
# HammingDistance_Accuracy(X_Validation_HD, Y_Validation, X_train_HD, Y_train)

In [25]:
def CalculateED(Dt,D):
    value = 0
    for i in range(len(D)):
        value += pow(Dt[i] - D[i], 2)
    value = math.sqrt(value)
    return value

In [12]:
def EuclideanDistance(Dt, X_train, Y_train):
    Euclidean_D = []
    for i in range(len(X_train)):
#         Euclidean_D.append(CalculateED(Dt, X_train[i]))
        Euclidean_D.append(distance.euclidean(Dt, X_train[i]))
        
    
    output = []
    for k in K:
    #find k minimum indices
        indices = sorted(range(len(Euclidean_D)), key = lambda sub: Euclidean_D[sub])[:k] 
        Y_output = []
        for i in indices:
            Y_output.append(Y_train[i])   
    
        #find higher frequency result
        output.append(max(set(Y_output), key = Y_output.count)) 
    return output

In [78]:
def EuclideanDistance_Accuracy(X_Validation_ED, Y_Validation, X_train_ED, Y_train):
    count = [0, 0, 0]
    for i in range(len(X_Validation_ED)):
        result = EuclideanDistance(X_Validation_ED[i], X_train_ED, Y_train)
        
        for k in range(len(K)):
            if Y_Validation[i] == result[k] :
                count[k] += 1
        
        print(i,count)
    
    for k in range(len(K)):
        print("Accuracy is: ", (count[k]/len(Y_Validation) * 100))

# HammingDistance(X_Validation_HD[0], X_train_HD, Y_train)
# print(Y_Validation[0])

# EuclideanDistance_Accuracy(X_Validation_ED, Y_Validation, X_train_ED, Y_train)

In [46]:
start = time.time()
print(EuclideanDistance(X_Validation_HD[0], X_train_HD, Y_train))
print(Y_Validation[0])
end = time.time()
print(end - start)

['Wood_Working', 'Anime', 'Coffee']
Coffee
29.052965879440308


In [7]:
number_of_documents = len(X_train_HD)
d_word = np.sum(X_train_HD.copy(), axis=0)
# print(d_word)
# print(len(d_word))
IDF = []
for i in range(len(d_word)):
    temp = math.log2(number_of_documents/d_word[i])
    if temp <= 0:
        IDF.append(0.0001)
    else:
        IDF.append(temp)

In [8]:
def setWeight(D, Dt):
    totalWords_D = sum(D)
    totalWords_Dt = sum(Dt)
    for i in range(len(D)):
        if D[i] != 0 :
            D[i] = (D[i]/totalWords_D) * IDF[i]
        if Dt[i] != 0:
            Dt[i] = (Dt[i]/totalWords_Dt) * IDF[i]
    return D,Dt

In [9]:
def setWeightOfDocument(D):
    totalWords_D = sum(D)
    for i in range(len(D)):
        if D[i] != 0 :
            D[i] = (D[i]/totalWords_D) * IDF[i]
    return D

def SetAllWeightOfSET(X_train):
#     count = 0
    for D in X_train:
        D = setWeightOfDocument(D)
#         count += 1
#         print(count)
    return X_train

In [10]:
X = SetAllWeightOfSET(deepcopy(X_train_ED))
# V = SetAllWeightOfSET(deepcopy(X_Validation_ED))
# print(X_train_ED[0])
# print(X[0])

In [10]:
def CosineTheta(D , Dt):
    return np.dot(D,Dt)/(norm(D)*norm(Dt))

In [11]:
def CosineSimilarity(Dt, X_train, Y_train):
    Cosine_values = []
#     output = []
    for D in X_train:
        Cosine_values.append(1-distance.cosine(Dt,D))

#     for k in K:
    indices = (sorted(range(len(Cosine_values)), key = lambda sub: Cosine_values[sub])[-5:])
    Y_output = []
    for i in indices:
        Y_output.append(Y_train[i])

    output= (max(set(Y_output), key = Y_output.count)) 
#     print(output)
    return output

In [12]:
def CosineValidation_Test(X_Validation_ED, Y_Validation, X_train_ED, Y_train):
#     count = [0,0,0]
    count = 0
    for i in range(len(X_Validation_ED)):
#         result = CosineSimilarity(X_Validation_ED[i], X_train_ED, Y_train)
        
#         for k in range(len(K)):
        if Y_Validation[i] ==  CosineSimilarity(X_Validation_ED[i], X_train_ED, Y_train):
            count+= 1
        
        print(i, count)
    
#     for k in range(len(K)):
#         print("Accuracy is: ", (count[k]/len(Y_Validation) * 100))
    
    return (count/len(Y_Validation) * 100)

# CosineValidation_Test(V, deepcopy(Y_Validation), X, deepcopy(Y_train))

In [17]:
start = time.time()
CosineSimilarity(deepcopy(X_Validation_ED[0]),deepcopy(X_train_ED), deepcopy(Y_train))
print(Y_Validation[0])
end = time.time()
print(end - start)

Coffee
85.88710808753967


In [13]:
def denominatorOfNB(Dt,alpha):
    summation = 0
    
    for i in range(len(TopicName)):
        summation += Prob_DT_CM(Dt,i, alpha)
    
    return summation

In [14]:
def Prob_Wj_Cm(word, m, alpha):
    
    frequencyOf_W_in_M = 0
    TotalWords_in_M = len(CM_list[m])
    
    for i in range(len(CM_list[m])):
        frequencyOf_W_in_M += CM_list[m][i].count(word)
        
    value = (frequencyOf_W_in_M + alpha)/ (TotalWords_in_M + alpha * len(FeatureSpace))
    
    if value == 0:
        print(m, "e jhamela ache ", value, "->", word)

    return value

In [15]:
def Prob_DT_CM(Dt, m, alpha):
    P_Dt_Cm = 1
    
    for i in range(len(Dt)):
        if(Dt[i] != 0):
            P_Dt_Cm *= Prob_Wj_Cm(FeatureSpace[i],m, alpha)
#             print(P_Dt_Cm)
    
#     print("probab of DT being in ", m, " is ", P_Dt_Cm)
    return P_Dt_Cm

In [16]:
def NaiveBayes(Dt, m, denom, alpha):
    value = (Prob_DT_CM(Dt,m,alpha) * (1/len(CM_list)))/ (denom + alpha*len(FeatureSpace))
    return value

In [259]:
# Probability = []
# denom = denominatorOfNB(X_Validation_ED[6])
# # for i in range(len(CM_list)):
# #     Probability.append(NaiveBayes(X_Validation_ED[6],i, denom))

# # # print(Probability)    
# # print(TopicName[Probability.index(max(Probability))][0])
# print(denom)

# print(len(X_Validation_HD[6]))
# # len(CM_list)

alpha = [2, 1, 1.5, 0.5, 0.2, 0.1, 0.07, 0.05, 0.01, 0.005] 
print(alpha)

[2, 1.5, 0.2]


In [17]:
def NB_precision(X_Validation_HD, Y_Validation, alpha):
    count = 0
    for itr in range(len(X_Validation_HD)):
        Probability = []
        denom = denominatorOfNB(X_Validation_HD[itr], alpha)
#         if denom == 0:
#             continue
        for i in range(len(CM_list)):
            Probability.append(NaiveBayes(X_Validation_HD[itr],i, denom, alpha))
        
        prediction = TopicName[Probability.index(max(Probability))][0]
        if prediction == Y_Validation[itr]:
            count += 1
#             print(count)
    
#     print("Accuracy is: ", (count/len(Y_Validation) * 100)) 
    return ((count/len(Y_Validation)) * 100)

# NB_precision(X_Validation_HD, Y_Validation, 0.01)



In [77]:
# for alpha in x:
#     NB_precision(X_Validation_HD, Y_Validation, alpha)
#     print(alpha)
#     print("--------------------------")

In [18]:
TopicName = []
x=[d.keys() for d in Training_set]
for keys in x:
    if list(keys) not in TopicName:
        TopicName.append(list(keys))

CM_list = []

for j in range(len(TopicName)):
    CM_list.append([])
    for i in range(len(Training_set)):
        if list(Training_set[i].keys()) == TopicName[j]:
            y = list(Training_set[i].values())[0]
            CM_list[j].append(y)
            
# len(CM_list[1])
# len(FeatureSpace)

In [19]:
def getTestValue(itr):
#     print(len(Test_set))
    X_test_HD = []
    X_test_ED = []
    Y_test = []

    # print(inputs)
    Test_Final = []
    for input_ in inputs:
#         print(input_)
        temp = [Test for Test in Test_set if list(Test.keys())[0] == input_ ]
        Test_Final += temp[itr*10:itr*10+10]

    for i in range(len(Test_Final)):
        Y_test.append(list(Test_Final[i].keys())[0])
        temp = list(Test_Final[i].values())[0]
        HD = []
        ED = []
        
        for i in range(len(FeatureSpace)):
            if FeatureSpace[i] in temp:
                HD.append(1)
            else:
                HD.append(0)
                
            ED.append(temp.count(FeatureSpace[i]))
                
        X_test_HD.append(HD)
        X_test_ED.append(ED)
        
    return X_test_HD, Y_test, X_test_ED

In [ ]:
for iteration in range(50):
    X_test_HD, Y_test, X_train_ED = getTestValue()
    
    with open('NaiveBayes.txt', 'a') as f:
        f.write("%s\n" %accuracy)

In [ ]:
for iteration in range(50):
    X_test_HD, Y_test, X_test_ED = getTestValue(iteration)
    print("TestData extracted")
    
    accuracy_NB = NB_precision(X_test_HD, Y_test, 0.005)
    print("Done iteration NB", iteration, accuracy_NB)
    with open('NaiveBayes.txt', 'a') as f:
        f.write("%s\n" %accuracy_NB)
    
    X_TEST = SetAllWeightOfSET(deepcopy(X_test_ED))
    print("X_Test weight set")
    accuracy_CS = CosineValidation_Test(X_TEST, deepcopy(Y_test), X, deepcopy(Y_train))
    print("Done iteration CS", iteration, accuracy_CS)    
    with open('CosineSim.txt', 'a') as f2:
        f2.write("%s\n" %accuracy_CS)

TestData extracted
Done iteration NB 0 86.36363636363636
X_Test weight set
0 0
1 1
2 2
3 3
4 4
5 5
6 5
7 6
8 7
9 8
10 9
11 10
12 11
13 12
14 13
15 14
16 15
17 16
18 17
19 18
20 19
21 20
22 21
23 22
24 22
25 23
26 23
27 24
28 25
29 26
30 27
31 28
32 29
33 30
34 31
35 32
36 33
37 34
38 35
39 36
40 36
41 37
42 38
43 38
44 39
45 40
46 40
47 41
48 42
49 43
50 44
51 45
52 46
53 47
54 48
55 49
56 50
57 51
58 52
59 53
60 54
61 55
62 56
63 56
